In [1]:
import numpy as np
import pandas as pd
import glob
import os
#for dirname, _, filenames in os.walk('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, SGD
import pandas as pd

In [2]:
Mild = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented/*.*')
Moderate = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented/*.*')
NonDemented = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented/*.*')
VeryMild = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented/*.*')


data = []
labels = []

for i in NonDemented:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(0)

for i in VeryMild:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(1)

for i in Mild:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(2)

for i in Moderate:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(3)

In [3]:
import glob
Mild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented/*.*')
Moderate_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented/*.*')
NonDemented_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented/*.*')
VeryMild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented/*.*')


data_test = []
labels_test  = []

for i in NonDemented_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(0)

for i in VeryMild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(1)

for i in Mild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(2)

for i in Moderate_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(3)

In [4]:
data = np.array(data)
labels = np.array(labels)

data_test = np.array(data_test)
labels_test = np.array(labels_test)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255,
                            rotation_range=45,
                            width_shift_range=.15,
                            height_shift_range=.15,
                            horizontal_flip=False)

train_data_gen = datagen.flow(
    x=data,
    y=labels,
    batch_size=64,
)

In [6]:
test_data_gen = datagen.flow(
    x=data_test,
    y=labels_test
)

In [7]:
from keras.utils import to_categorical

train_data_gen.y = to_categorical(train_data_gen.y, num_classes=4)
test_data_gen.y = to_categorical(test_data_gen.y, num_classes=4)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [9]:
import keras
import tensorflow_addons as tfa
METRICS = [keras.metrics.CategoricalAccuracy(name='accuracy'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.AUC(name='auc'),
           tfa.metrics.F1Score(num_classes=4),
           #tfa.metrics.MultiLabelConfusionMatrix(num_classes=4, name='confusion_matrix')
          ]

/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

In [11]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

# Train the model with the metrics callback
history = model.fit(train_data_gen.x, train_data_gen.y, batch_size=128, epochs=25, validation_data=(test_data_gen.x, test_data_gen.y))


Epoch 1/25
41/41 [==============================] - 228s 6s/step - loss: 20.2483 - accuracy: 0.4351 - precision: 0.4199 - recall: 0.1888 - auc: 0.6883 - f1_score: 0.2300 - val_loss: 1.0621 - val_accuracy: 0.5004 - val_precision: 0.6641 - val_recall: 0.0665 - val_auc: 0.7898 - val_f1_score: 0.1668
Epoch 2/25
41/41 [==============================] - 231s 6s/step - loss: 1.0545 - accuracy: 0.4915 - precision: 0.5682 - recall: 0.2531 - auc: 0.7874 - f1_score: 0.2267 - val_loss: 1.0200 - val_accuracy: 0.5027 - val_precision: 0.5530 - val_recall: 0.4159 - val_auc: 0.8053 - val_f1_score: 0.1713
Epoch 3/25
41/41 [==============================] - 233s 6s/step - loss: 1.0125 - accuracy: 0.5198 - precision: 0.6046 - recall: 0.2951 - auc: 0.8060 - f1_score: 0.2865 - val_loss: 0.9749 - val_accuracy: 0.5403 - val_precision: 0.6310 - val_recall: 0.3503 - val_auc: 0.8180 - val_f1_score: 0.2727
Epoch 4/25
41/41 [==============================] - 217s 5s/step - loss: 0.9384 - accuracy: 0.5460 - precisi